# Using MLRUN: A getting started example
### Running the code in your notebook

In [1]:
# nuclio: start-code

In [34]:
from mlrun import mlconf,get_or_create_ctx, code_to_function, mount_v3io
from mlrun.artifacts import TableArtifact

## Change the IP to your PC's IP address

In [35]:
mlconf.dbpath = 'http://mlrun-api:8080'

In [36]:
# Jobs running with mlrun have access to a context that provides
# information about the execution of a job
# This context is available for a single job or a step in pipeline
mlrun_context = get_or_create_ctx('my_job')


[mlrun] 2020-03-11 14:48:34,188 logging run results to: http://mlrun-api:8080


## Incorporating mlrun into your code to keep track of jobs

In [37]:
def my_job(context):
       
    # Get parameters
    # With the context in place, mlrun keeps track of any parameters
    # you used to execute your job 
    param1 = mlrun_context.get_param('param1',4)
    
    # Save results to mlrun
    mlrun_context.log_result("Calulated_Value",param1*3)
    
    # Save artifacts from your run
    # This functionality supports Plots, tables, models
    # For this example we will save a table
    mlrun_context.log_result('accuracy',0.8)
    mlrun_context.log_artifact(TableArtifact('dataset', '1,2,3\n4,5,6\n',
                                         header=['A', 'B', 'C'], target_path='dataset.csv'))

In [38]:
# nuclio: end-code

In [41]:
newfn = code_to_function('my_job',kind='job',filename="/User/mlrun_blog/containerazing.ipynb").apply(mount_v3io())

In [42]:
newfn.spec.image = "mlrun/mlrun"
#newfn.build_config(image='test/img')

In [43]:
print(newfn.to_yaml())

kind: job
metadata:
  name: my-job
  tag: ''
  project: ''
spec:
  command: ''
  args: []
  image: mlrun/mlrun
  volumes:
  - flexVolume:
      driver: v3io/fuse
      options:
        accessKey: e4d2c610-4059-4ed3-9727-e45b073b3869
        container: users
        subPath: /marcelo
    name: v3io
  volume_mounts:
  - mountPath: /User
    name: v3io
  env:
  - name: V3IO_API
    value: v3io-webapi.default-tenant.svc:8081
  - name: V3IO_USERNAME
    value: marcelo
  - name: V3IO_ACCESS_KEY
    value: e4d2c610-4059-4ed3-9727-e45b073b3869
  description: ''
  build:
    functionSourceCode: IyBHZW5lcmF0ZWQgYnkgbnVjbGlvLmV4cG9ydC5OdWNsaW9FeHBvcnRlciBvbiAyMDIwLTAzLTExIDE0OjQ5Cgpmcm9tIG1scnVuIGltcG9ydCBtbGNvbmYsZ2V0X29yX2NyZWF0ZV9jdHgsIGNvZGVfdG9fZnVuY3Rpb24sIG1vdW50X3YzaW8KZnJvbSBtbHJ1bi5hcnRpZmFjdHMgaW1wb3J0IFRhYmxlQXJ0aWZhY3QKCm1sY29uZi5kYnBhdGggPSAnaHR0cDovL21scnVuLWFwaTo4MDgwJwoKbWxydW5fY29udGV4dCA9IGdldF9vcl9jcmVhdGVfY3R4KCdteV9qb2InKQoKZGVmIG15X2pvYihjb250ZXh0KToKICAgICAgIAogICAgcGFyYW0

In [44]:
train_run = newfn.run(handler="my_job",out_path="v3io:///bigdata/mlrund/db",params={'param1': 5})

[mlrun] 2020-03-11 14:49:13,237 starting run my-job-my_job uid=0777242197914fdf9a6125b22e688d5c  -> http://mlrun-api:8080
[mlrun] 2020-03-11 14:49:13,432 Job is running in the background, pod: my-job-my-job-8czsc
[mlrun] 2020-03-11 14:49:18,892 logging run results to: http://mlrun-api:8080
[mlrun] 2020-03-11 14:49:19,167 log artifact dataset at v3io:///bigdata/mlrund/db/dataset, size: 12, db: Y

[mlrun] 2020-03-11 14:49:19,218 run executed, status=completed
final state: succeeded


/User/.pythonlibs/lib/python3.6/site-packages/mlrun/render.py:337: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...688d5c,0,Mar 11 14:49:18,completed,my-job-my_job,host=my-job-my-job-8czsckind=jobowner=admin,,param1=5,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 0777242197914fdf9a6125b22e688d5c  , !mlrun logs 0777242197914fdf9a6125b22e688d5c 
[mlrun] 2020-03-11 14:49:22,864 run executed, status=completed
